<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/613_GCOv2_MVPv2_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Governance & Compliance Orchestrator — v2 (MVPv2)

## What v2 Adds Over v1

| Dimension | V1 | V2 |
|-----------|----|-----|
| **Scope** | Single-run snapshot | Multiple agents, historical runs (portfolio summary, bias/drift history) |
| **Risk** | Basic policy checks, violations, scores | Evolving risk, trend analysis, early-warning flags |
| **Bias/Drift** | Current signals only | Bias/drift history, trend_direction, delta vs prior |
| **Human layer** | Implicit | Human overrides, policy enforcement events, time-to-resolution |
| **Regulation** | Policy rules | Cases, root-cause summary, regulatory exposure (USD) |
| **Executive view** | Summary stats | Portfolio rollup, executive triggers, ROI vs risk readiness |

**From "auditor bot" → "enterprise AI risk command center."**

---

## Data → State Mapping

| Data file | State field(s) | Notes |
|-----------|----------------|--------|
| `agent_action_logs_batch_*.json`, `agent_action_logs_day_02.json` | `agent_action_logs`, `events_lookup` | Merged; day_02 has cost_usd, latency_ms, environment, policy_candidates |
| `policy_rules.json` | `policy_rules`, `policy_lookup` | Conditions, severity, required_action |
| `bias_signals.json` | `bias_signals` | Wrapper normalized to list of signals |
| `drift_and_degradation_signals.json` | `drift_signals` | Same |
| `policy_enforcement_events.json` | `policy_enforcement_events` | Override, approver_role, resolution_time_minutes |
| `governance_cases.json` | `governance_cases`, `cases_lookup` | remediation_steps, financial_exposure_usd |
| `bias_signals_history.json` | `bias_signals_history` | Per-run snapshots, trend_direction |
| `drift_signals_history.json` | `drift_signals_history` | Per-run drift |
| `governance_portfolio_summary.json` | `governance_portfolio_summary` | overall_risk_score, risk_trend, open_cases, executive_summary |

---

## New State Fields (v2)

- **policy_enforcement_events**, **governance_cases** — Human escalation and case tracking  
- **bias_signals_history**, **drift_signals_history**, **governance_portfolio_summary** — Time-series for trends  
- **risk_trends** — trend_direction, delta_vs_prior, early_warning_flags  
- **portfolio_rollup** — high_risk_agents, open_cases, regulatory_exposure_usd, top_root_causes  
- **executive_triggers** — Conditions that warrant exec attention (risk threshold, open cases, regulatory exposure)  
- **executive_escalation_triggered** — True when the conditional escalation path was taken  
- **escalation_alerts** — Alerts created when escalation runs (from toolshed.alerting)  
- **root_cause_summary** — Top root causes with agent/case linkage  

---

## What Makes v2 Materially Stronger

1. **Portfolio-level view** — One rollup across all agents and cases; exec sees high-risk agents, open cases, exposure.  
2. **Trends** — Risk trend (improving/stable/worsening), delta vs prior run, early-warning flags.  
3. **Human-in-the-loop visibility** — Enforcement events and overrides with resolution time; cases with remediation and financial exposure.  
4. **Executive triggers** — Automatic flags when risk score, open cases, or regulatory exposure exceed thresholds.  
5. **Root-cause tracing** — Top root causes linked to agents and cases for audit and remediation.  

---

## Implemented (MVP v2 + enhancements)

**Core:**  
- Rule-based policy evaluation and risk scoring  
- All v2 data sources loaded and used  
- Portfolio rollup, trends, executive triggers, root-cause summary  
- LangGraph linear workflow with **conditional escalation** (when triggers fire → escalation_alert → then prioritize → report)  
- Report to file: `output/governance_compliance_reports/gco_audit_v2_<timestamp>.md`  

**CEO & business manager:**  
- **Target + on track** — Configurable target (e.g. overall risk < 60); report shows "on track" or "above target".  
- **One-view** — Headline (Governance risk: HIGH/MEDIUM/LOW), one-line "So what?", 1–3 action-oriented next steps (assign cases, schedule review).  
- **Segment view by agent** — Table with risk score, status, exposure (USD), **R/Y/G level** (critical / elevated / monitor).  
- **Root-cause → case linkage** — Top root causes linked to agent and case ID (e.g. residual_marketing_bias → MarketingAgent, CASE_9005).  
- **Regulatory context** — Frameworks implicated (e.g. EU_AI_ACT, EEOC) from bias/drift signals.  
- **Alerting on escalation** — When the escalation path runs, toolshed alerting creates an executive-attention alert (stored in state; can be logged or sent to a channel).  

**Quality:**  
- **Tests** — 12 tests (unit: verdict, R/Y/G, next steps, regulatory frameworks, root-cause linkage; integration: full run, state shape). Run: `pytest test_governance_compliance_v2.py -v`.  
- **Datetime** — Timezone-aware timestamps (no deprecation warnings).  

---

## Run

```bash
python run_governance_compliance_v2.py
```

Report: `output/governance_compliance_reports/gco_audit_v2_<timestamp>.md`

**Tests:** `pytest test_governance_compliance_v2.py -v`

---

## V3 / future (optional)

- LLM-generated violation explanations (config flag exists; can be wired in)  
- Time-window filtering (e.g. last 7 days) in loader  
- PDF/HTML one-pager via toolshed when the use case needs it
